In [25]:
#!pip install pandas
#!pip install matplotlib

In [28]:
#!pip install fsspec

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import findspark

In [3]:
findspark.init()

In [4]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2022-10-13 12:12:20,344 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2022-10-13 12:12:25,795 WARN yarn.Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [10]:
data = spark.read.csv("hdfs:///user/ubuntu/ec2_data_folder/data/01.csv", header='true' , inferSchema='true', )

In [11]:
data.show(5)

+-----------------------------+----+-----------------------+----------+---------+----------+---------+---+
|2009-01-09 16:10:00.000000170|31.9|2009-01-09 16:10:00 UTC|-73.873027|40.773883|-73.984545|40.769545|  3|
+-----------------------------+----+-----------------------+----------+---------+----------+---------+---+
|         2009-01-09 15:01:...| 5.7|   2009-01-09 15:01:...|-73.992826|40.751271|-73.982786|40.742432|  1|
|         2009-01-25 18:41:...| 6.1|   2009-01-25 18:41:...|-74.000122| 40.74308| -74.00615|40.734233|  5|
|         2009-01-16 21:10:...| 4.5|   2009-01-16 21:10:...|-73.994143|40.751193|-73.988897|40.759742|  1|
|         2009-01-24 09:51:...| 5.3|   2009-01-24 09:51:...| -73.96707|40.769178|-73.954895| 40.77821|  1|
|         2009-01-23 07:28:...| 8.1|   2009-01-23 07:28:...|-73.996068|40.732605|-73.980675|40.761862|  1|
+-----------------------------+----+-----------------------+----------+---------+----------+---------+---+
only showing top 5 rows



In [12]:
type(data)

pyspark.sql.dataframe.DataFrame

In [13]:
# out of memory...
# df = data.toPandas()
# print(df)

In [14]:
# df.describe()

In [15]:
data.printSchema()

root
 |-- 2009-01-09 16:10:00.000000170: string (nullable = true)
 |-- 31.9: double (nullable = true)
 |-- 2009-01-09 16:10:00 UTC: string (nullable = true)
 |-- -73.873027: double (nullable = true)
 |-- 40.773883: double (nullable = true)
 |-- -73.984545: double (nullable = true)
 |-- 40.769545: double (nullable = true)
 |-- 3: integer (nullable = true)



In [16]:
# data가 너무 크니 1000개로 자르기
little_data = data.limit(10000)

In [17]:
# pyspark.sql.dataframe.DataFrame to Pandas df
df = little_data.toPandas()
df.head()

,2009-01-09 16:10:00.000000170,31.9,2009-01-09 16:10:00 UTC,-73.873027,40.773883,-73.984545,40.769545,3
0,2009-01-09 15:01:20.0000002,5.7,2009-01-09 15:01:20 UTC,-73.992826,40.751271,-73.982786,40.742432,1
1,2009-01-25 18:41:00.000000193,6.1,2009-01-25 18:41:00 UTC,-74.000122,40.743080,-74.006150,40.734233,5
2,2009-01-16 21:10:00.00000056,4.5,2009-01-16 21:10:00 UTC,-73.994143,40.751193,-73.988897,40.759742,1
3,2009-01-24 09:51:00.000000115,5.3,2009-01-24 09:51:00 UTC,-73.967070,40.769178,-73.954895,40.778210,1
4,2009-01-23 07:28:00.0000005,8.1,2009-01-23 07:28:00 UTC,-73.996068,40.732605,-73.980675,40.761862,1


In [18]:
df.describe()

,31.9,-73.873027,40.773883,-73.984545,40.769545,3
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,9.580576,-72.857598,40.138005,-72.886298,40.154363,1.690900
std,7.622211,9.021849,4.970477,8.903243,4.905219,1.248564
min,2.500000,-74.690055,0.000000,-74.690003,0.000000,1.000000
25%,5.400000,-73.991393,40.736458,-73.990757,40.734746,1.000000
50%,7.300000,-73.981269,40.753602,-73.979805,40.753363,1.000000
75%,10.500000,-73.967441,40.767862,-73.964150,40.769223,2.000000
max,100.000000,0.000000,45.035255,0.000000,45.035269,6.000000


In [19]:
#check 결측치
df.isnull().sum().sort_values(ascending=False)
print(df.shape)

(10000, 8)


In [20]:
#drop 결측치
df = df.drop(df[df.isnull().any(1)].index, axis = 0)

/tmp/ipykernel_1925303/2820316284.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.any and Series.any will be keyword-only.
  df = df.drop(df[df.isnull().any(1)].index, axis = 0)


In [21]:
# #fare_amount값이 0인 경우를 찾기 :2가지
# from collections import Counter
# Counter(df['fare_amount']<0)

KeyError: 'fare_amount'

In [ ]:
#drop fare_amount가 0인경우 (요금이 0이 나온경우는 제거)
df = df.drop(df[df['fare_amount']<0].index, axis=0)
df.shape

In [ ]:
df['key'] = pd.to_datetime(df['key'])
df['pickup_datetime']  = pd.to_datetime(df['pickup_datetime'])

In [ ]:
#check the dtypes after conversion
df.dtypes

In [ ]:
plt.figure(figsize=(15,7))
plt.hist(df['passenger_count'], bins=15)
plt.xlabel('No. of Passengers')
plt.ylabel('Frequency')

In [ ]:
# s3://team07-data/SPARK-ETL/

df.write.format('csv').option('header','true').save('s3://team07-data/SPARK-ETL/test.csv',mode='overwrite')